In [5]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override = True)

True

The first type of MCP Server: runs locally, everything local
Here's a really interesting one: a knowledge-graph based memory.

It's a persistent memory store of entities, observations about them, and relationships between them.

https://github.com/modelcontextprotocol/servers/tree/main/src/memory

In [6]:
params = {"command": "npx","args": ["-y", "mcp-memory-libsql"],"env": {"LIBSQL_URL": "file:./memory/ed.db"}}

async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()

mcp_tools

Error initializing MCP server: Timed out while waiting for response to ClientRequest. Waited 60.0 seconds.
Error cleaning up server: unhandled errors in a TaskGroup (1 sub-exception)


McpError: Timed out while waiting for response to ClientRequest. Waited 60.0 seconds.

In [2]:
from accounts import Account

In [ ]:
account = Account.get("Ellen")
account

Account(name='ellen', balance=10000.0, strategy='', holdings={}, transactions=[], portfolio_value_time_series=[])

In [4]:
account.buy_shares("AMZN", 3, "Because this bookstore website looks promising")

'Completed. Latest details:\n{"name": "ellen", "balance": 9358.00858, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 213.99714, "timestamp": "2025-06-09 13:29:58", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-06-09 13:29:58", 9998.71858]], "total_portfolio_value": 9998.71858, "total_profit_loss": -1.2814199999993434}'

In [5]:
account.report

<bound method Account.report of Account(name='ellen', balance=9358.00858, strategy='', holdings={'AMZN': 3}, transactions=[3 shares of AMZN at 213.99714 each.], portfolio_value_time_series=[('2025-06-09 13:29:58', 9998.71858)])>

In [ ]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 3,
  'price': 213.99714,
  'timestamp': '2025-06-09 13:29:58',
  'rationale': 'Because this bookstore website looks promising'}]

## write an MCP server and use it directly!

In [7]:
#now i will be using accounts server as MCP server

params = {"command":"uv", "args":["run", "accounts_server.py"]}
async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()

In [8]:
mcp_tools

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None),
 Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None),
 Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', '

## Perfect! Now simply use the tool 

In [10]:
instructions = " You are able to manage an account for a client, and answer questions about the account."
request = "My name is Ellen and my account is under the name Ellen. What's my balance and my holdings?"
model = "gpt-4o-mini"

In [11]:
async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(
        name = "account_manager",
        instructions=instructions,
        model = model, 
        mcp_servers=[mcp_server]
    )
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Your account balance is **$9,358.01**. 

You currently hold **3 shares** of **Amazon (AMZN)**.

### Now let's build our own MCP Client

In [12]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None), Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None), Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', 'ty

In [13]:
#use the tool

agent= Agent(
    name="account_manager",
    instructions=instructions,
    model=model,
    tools=openai_tools)
result = await Runner.run(agent, request)
display(Markdown(result.final_output))

Your current balance is **$9,358.01**. 

As for your holdings, you own **3 shares of Amazon (AMZN)**.

In [14]:
context = await read_accounts_resource("ellen")
print(context)

{"name": "ellen", "balance": 9358.00858, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 213.99714, "timestamp": "2025-06-09 13:29:58", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-06-09 13:29:58", 9998.71858], ["2025-06-09 13:31:05", 9998.71858]], "total_portfolio_value": 9998.71858, "total_profit_loss": -1.2814199999993434}


In [ ]:
from accounts import Account
Account.get("ellen").report()

'{"name": "ellen", "balance": 9358.00858, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 213.99714, "timestamp": "2025-06-09 13:29:58", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-06-09 13:29:58", 9998.71858], ["2025-06-09 13:31:05", 9998.71858], ["2025-06-09 13:31:07", 9998.71858]], "total_portfolio_value": 9998.71858, "total_profit_loss": -1.2814199999993434}'